<a href="https://colab.research.google.com/github/florenceyuen/sit-to-stand-tracker/blob/main/sit_to_stand_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check gpu connections
!nvidia-smi

Wed Jun  4 22:01:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
# Yolo and dependency installations
!pip install numpy==1.24.3 --quiet

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.7/112.6 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
model = YOLO(f'{HOME}/yolov8s-seg.pt')
#results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)

100%|██████████| 22.8M/22.8M [00:00<00:00, 398MB/s]


In [ ]:
%%html
<video id="video" autoplay playsinline style="display:none;"></video>
<canvas id="canvas" style="display:none;"></canvas>
<script>
(async () => {
  const video  = document.getElementById('video');
  const canvas = document.getElementById('canvas');
  const ctx    = canvas.getContext('2d');

  try {
    const stream = await navigator.mediaDevices.getUserMedia({video:true});
    video.srcObject = stream;
    await video.play();
  } catch (e) {
    console.error('Camera access denied or not available', e);
    return;
  }

  canvas.width  = video.videoWidth;
  canvas.height = video.videoHeight;
  google.colab.output.setIframeHeight(canvas.height + 20);

  window.captureFrame = () => {
    ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
    return canvas.toDataURL('image/jpeg', 0.8);
  };
</script>


In [ ]:
# Adding yolo pose models
from ultralytics import YOLO
pose_model = YOLO("yolov8n-pose.pt")  # or yolov8m-pose.pt for better accuracy
# pose_model = YOLO('yolov8m-pose.pt')


100%|██████████| 6.52M/6.52M [00:00<00:00, 183MB/s]


## Ultralytics YOLOv8-Pose follows the 17-keypoint “COCO” convention
---

## 1. How the angle calculation works

1. **Gather the three 2-D points** that form the joint:  
   * **S** (start / proximal …)  
   * **E** (elbow – vertex)  
   * **W** (end / distal …)

2. **Form two limb-segment vectors** that share the vertex  

$$
\vec{v_1} = \mathbf{S} - \mathbf{E},\qquad
\vec{v_2} = \mathbf{W} - \mathbf{E}
$$


3. **Compute the cosine**  

$$
\cos\theta = \frac{\,\vec{v_1}\cdot\vec{v_2}\,}
                 {\lVert\vec{v_1}\rVert\,\lVert\vec{v_2}\rVert}
$$


4. **Convert to degrees**  


$$
\theta = \arccos(\cos\theta)\times\frac{180}{\pi}
$$

The tiny `1e-6` in the denominator of `angle_at_joint()` avoids division-by-zero.

---

## 2. YOLOv8 / COCO 17-keypoint index map

| Index | Landmark | Typical use-case | Notes |
|-----:|-----------|------------------|-------|
| 0 | Nose | Head orientation | Center of face |
| 1 | Left Eye | Gaze / blink | |
| 2 | Right Eye | | |
| 3 | Left Ear | Head yaw | |
| 4 | Right Ear | | |
| **5** | **Left Shoulder** | Upper-arm root | Start of **left elbow angle** |
| **6** | **Right Shoulder** | | Start of **right elbow angle** |
| **7** | **Left Elbow** | Elbow flexion | Vertex |
| **8** | **Right Elbow** | | Vertex |
| 9 | Left Wrist | Wrist pose / elbow | |
| 10 | Right Wrist | | |
| 11 | Left Hip | Trunk inclination | |
| 12 | Right Hip | | |
| 13 | Left Knee | Knee flexion | |
| 14 | Right Knee | | |
| 15 | Left Ankle | Gait analysis | |
| 16 | Right Ankle | | |

### Example – elbow flexion  
* **Left:** `[5 → 7 → 9]` (shoulder–elbow–wrist)  
* **Right:** `[6 → 8 → 10]`

---

## 3. Confidence filtering & best practices

* **Keypoint confidence**: use the 3ʳᵈ value in each keypoint row (0–1) to skip uncertain detections.  
* **Units**: coordinates are pixels; angles are unit-free, but segment *lengths* need scaling.  
* **2-D vs 3-D**: for true joint angles in space, capture depth or multi-view 3-D first.

---

## 4. Further documentation & resources

* **Ultralytics Pose guide** – quick-start, skeleton diagram.  
https://docs.ultralytics.com/tasks/pose/#models

https://github.com/Alimustoofaa/YoloV8-Pose-Keypoint-Classification


In [ ]:
os.kill(os.getpid(), 9)

In [ ]:
from itertools import cycle
"""
Monitors and evaluates sit-to-stand movements, commonly used to
assess lower-body strength and mobility, particularly in older adults. The application
should track key joint angles or movement patterns to detect transitions and provide
feedback on performance quality
"""

# ─── One‑Cell Live Pose + Elbow‑Angle Estimation (fixed keypoint extraction) ───
from IPython.display import Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import cv2, numpy as np, time
import ipywidgets as widgets
from ultralytics import YOLO
import time

# Global variables for timing
cycle_start_time = None
rep_durations = []

# 1) Inject JS for webcam + captureFrame()
display(Javascript("""
(async () => {
  const v = document.createElement('video');
  v.autoplay = v.playsInline = true;
  v.style.display = 'none';
  document.body.appendChild(v);

  const c = document.createElement('canvas');
  c.style.display = 'none';
  document.body.appendChild(c);
  const ctx = c.getContext('2d');

  const stream = await navigator.mediaDevices.getUserMedia({video:true});
  v.srcObject = stream;
  await v.play();

  c.width = v.videoWidth; c.height = v.videoHeight;
  google.colab.output.setIframeHeight(c.height + 20);

  window.captureFrame = () => {
    ctx.drawImage(v, 0, 0, c.width, c.height);
    return c.toDataURL('image/jpeg', 0.8);
  };
})();
"""))

# 2) Wait for the JS function to be ready
print("⏳ Initializing camera…")
for _ in range(25):
    try:
        if eval_js("typeof captureFrame") == 'function':
            print("✅ Camera ready!")
            break
    except Exception:
        pass
    time.sleep(0.2)
else:
    raise RuntimeError("Camera never initialized—did you allow access?")

# 3) Helpers to grab frames and compute angles
def get_frame():
    data = eval_js('captureFrame()')
    _, b64 = data.split(',', 1)
    arr = np.frombuffer(b64decode(b64), dtype=np.uint8)
    return cv2.imdecode(arr, cv2.IMREAD_COLOR)

def angle_at_joint(kps, si, ei, wi):
    S, E, W = kps[si, :2], kps[ei, :2], kps[wi, :2]
    v1, v2 = S - E, W - E
    cosang = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2) + 1e-6)
    return np.degrees(np.arccos(np.clip(cosang, -1, 1)))

def angle_visualization(vis, kps, landmark, angle):
    """
    Overlay angles onto the image visualization at the specified landmark

    - vis: image (numpy array) to draw on
    - kps: keypoints numpy array (shape [17,3])
    - landmark: int, index of the keypoint to annotate
    - angle: float, the angle value to display
    """
    pt_joint = tuple(kps[landmark, :2].astype(int))
    cv2.putText(vis, f"{int(angle)}°", (pt_joint[0] - 20, pt_joint[1]),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

def transition_detection(hip_angle, knee_angle, prev_posture):
  STANDING_THRESHOLD = 140 # standing threshold: hip, knee angle > 140 deg
  SITTING_THRESHOLD = 40 # sitting threshold: hip, knee angle < 100 deg

  SITTING_THRESHOLD_HIP = 30
  SITTING_THRESHOLD_KNEE = 140

  STANDING_THRESHOLD_HIP = 20
  STANDING_THRESHOLD_KNEE = 160

  # Detect when posture transitions from sitting to standing, or standing to sitting
  if (hip_angle < STANDING_THRESHOLD_HIP) and (knee_angle > STANDING_THRESHOLD_KNEE):
    posture = "standing"
  elif (hip_angle < SITTING_THRESHOLD_HIP) and (knee_angle < SITTING_THRESHOLD_KNEE):
      posture = "sitting"
  else: # in between state
    posture = prev_posture
  return posture

# Symmetry Index: |L-R|/((L+R)/2) *100%, good symmetry is <10%
def eval_symmetry(left, right):
  imbalance = abs(left-right)
  symmetry = (imbalance/((left+right)/2))*100
  return symmetry

def calc_ROM(hip_angles_left, hip_angles_right, knee_angles_left, knee_angles_right):
  """ Range of motion (ROM) during transitions"""
  rom_hip_left = max(hip_angles_left) - min(hip_angles_left)
  rom_hip_right = max(hip_angles_right) - min(hip_angles_right)
  rom_knee_left = max(knee_angles_left) - min(knee_angles_left)
  rom_knee_right = max(knee_angles_right) - min(knee_angles_right)

  return rom_hip_left, rom_hip_right, rom_knee_left, rom_knee_right

def calc_cycle_time(cycle_start_time):
  if cycle_start_time:
      rep_duration = time.time() - cycle_start_time
      rep_durations.append(rep_duration)
      print(f"Cycle completed in {rep_duration:.2f} seconds")

def track_cycles(hip_angle, knee_angle, prev_posture, num_reps, cycle_start_time):
  """ Count number of cycles completed, track cycle timing """
  current_posture = transition_detection(hip_angle, knee_angle, prev_posture)

  cycle_completed = False
  rep_duration = None

  if prev_posture == 'sitting' and current_posture == 'standing': # user begins standing
    cycle_start_time = time.time()  # Cycle started, start timing
    print("Changed to standing")
  # Check if completed one sit-to-stand-to-sit cycle
  elif prev_posture == 'standing' and current_posture == 'sitting':
    num_reps += 1
    cycle_completed = True

    # Claculate total cycle duration and reset for next cycle
    calc_cycle_time(cycle_start_time)
    cycle_start_time = None

  return current_posture, num_reps, cycle_completed, cycle_start_time, rep_duration

def compute_velocity(angle_list, fps=30):
    velocities = []
    for i in range(1, len(angle_list)):
        vel = (angle_list[i] - angle_list[i-1]) * fps  # degrees per second approx.
        velocities.append(vel)
    return velocities

def compute_smoothness(velocities):
    # Lower std dev means smoother movement
    return np.std(velocities)

# 4) Load YOLOv8‑Pose model
pose_model = YOLO("yolov8n-pose.pt")  # or your custom pose weights

# 5) Create a persistent widget for display
img_wid = widgets.Image(format='jpeg')
display(img_wid)

# 6) Live loop: predict, draw skeleton, compute & overlay sitting and standing angles
print("▶️ Live pose + sit-stand-tracker—Interrupt (⏹) to stop.")

# track number of repetitions (sit-->stand-->sit)
hip_angles_left, hip_angles_right = [], []
knee_angles_left, knee_angles_right = [], []
prev_posture = None
num_reps = 0
try:
    while True:
        frame = get_frame()
        res   = pose_model.predict(frame, stream=False)[0]
        vis   = res.plot()

        # **Fix**: extract the raw tensor from Coordinates, then to numpy
        kps_arr = res.keypoints.data.detach().cpu().numpy()  # shape (n_people, 17, 3)

        if kps_arr.shape[0] == 0:
          continue  # skip frame if no people are detected

        for kps in kps_arr:
            # Filter low-confidence
            if kps.shape[0] < 17: # skip incomplete detections for keypoints shape
                continue  # skip incomplete detections

            # Skip low-confidence keypoints by checking visibility/ confidence for hip, knee, ankle points
            confidence_threshold = 0.3
            required_indices = [5, 6, 11, 12, 13, 14, 15, 16 ]  # hips, knees, ankles, elbows: # , 7, 8
            if any(kps[i, 2] < confidence_threshold for i in required_indices):
                continue

            # angles for left and right knee movement
            kL = angle_at_joint(kps, 11, 13, 15)
            kR = angle_at_joint(kps, 12, 14, 16)

            # angles for left and right hip movement (shoulder, hip, knee)
            hip_left = angle_at_joint(kps, 11, 5, 13)
            hip_right = angle_at_joint(kps, 12, 6, 14)

            print("Angle:", hip_left, hip_right, kL, kR)

            # Elbow: left = [5→7→9], right = [6→8→10]
            # aL = angle_at_joint(kps, 5, 7, 9)
            # aR = angle_at_joint(kps, 6, 8, 10

            # Add angles for calculating range of motion
            hip_angles_left.append(hip_left)
            hip_angles_right.append(hip_right)
            knee_angles_left.append(kL)
            knee_angles_right.append(kR)

            # Determine posture and sit-stand transitions
            avg_hip = (hip_left + hip_right)/2
            avg_knee = (kL + kR)/2

            hip_avg = (hip_left + hip_right) / 2
            knee_avg = (kL + kR) / 2

            print(f"Hip avg: {hip_avg:.1f}, Knee avg: {knee_avg:.1f} => Posture: {prev_posture}")

            # Determine current posture and update rep count using your function
            prev_posture, num_reps, cycle_completed, cycle_start_time, rep_duration = track_cycles(hip_avg, knee_avg, prev_posture, num_reps, cycle_start_time)

            if cycle_completed:
              # Calculate ROM and symmetry
              rom_hip_L, rom_hip_R, rom_knee_L, rom_knee_R = calc_ROM(hip_angles_left, hip_angles_right, knee_angles_left, knee_angles_right)

              sym_hip = eval_symmetry(rom_hip_L, rom_hip_R)
              sym_knee = eval_symmetry(rom_knee_L, rom_knee_R)

              track_cycles(hip_left, kL, prev_posture, num_reps, cycle_start_time)
              track_cycles(hip_right, kR, prev_posture, num_reps, cycle_start_time)

              angle_lists = [hip_angles_left, hip_angles_right, knee_angles_left, knee_angles_right]
              smoothness_results = []

              for angles in angle_lists:
                  # Calculate smoothness using velocity for each angle
                  velocities = compute_velocity(angles)
                  smoothness = compute_smoothness(velocities)
                  smoothness_results.append(smoothness)

                  # Clear angle lists for next cycle rep
                  angles.clear()

              smooth_hip_L, smooth_hip_R, smooth_knee_L, smooth_knee_R = smoothness_resultss

              # Visualization and overlay of info for number of reps, symmetry, smoothness
              cv2.putText(vis, f"Reps: {num_reps}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
              cv2.putText(vis, f"Hip Sym: {sym_hip:.1f}%", (10,70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0) if sym_hip < 10 else (0,0,255), 2)
              cv2.putText(vis, f"Knee Sym: {sym_knee:.1f}%", (10,100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0) if sym_knee < 10 else (0,0,255), 2)

              cv2.putText(vis, f"Rep Time: {rep_duration:.2f}s" if rep_duration else "Rep Time: --", (10,130), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
              cv2.putText(vis, f"Hip L Smoothness: {smooth_hip_L:.1f}", (10,160), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
              cv2.putText(vis, f"Hip R Smoothness: {smooth_hip_R:.1f}", (10,160), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
              cv2.putText(vis, f"Knee R Smoothness: {smooth_knee_L:.1f}", (10,160), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
              cv2.putText(vis, f"Knee Smoothness: {smooth_knee_L:.1f}", (10,160), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

            # Landmark index and corresponding angle value for visualization
              landmarks_angles = [
                  (13, kL),          # left knee angle
                  (14, kR),          # right knee angle
                  (5, hip_left),     # left hip angle
                  (6, hip_right)     # right hip angle
                  # (7, aL),           # left elbow angle
                  # (8, aR)            # right elbow angle
              ]

              for landmark, angle in landmarks_angles:
                  angle_visualization(vis, kps, landmark, angle)

        _, jpg = cv2.imencode('.jpg', vis)
        img_wid.value = jpg.tobytes()
        time.sleep(0.03)  # tune this for latency vs. CPU/GPU load

except KeyboardInterrupt:
    print("⏹ Segmentation stopped.")


<IPython.core.display.Javascript object>

⏳ Initializing camera…
✅ Camera ready!


Image(value=b'', format='jpeg')

▶️ Live pose + sit-stand-tracker—Interrupt (⏹) to stop.

0: 480x640 1 person, 11.0ms
Speed: 1.4ms preprocess, 11.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.1ms
Speed: 1.7ms preprocess, 8.1ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.0ms
Speed: 1.8ms preprocess, 11.0ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.6ms
Speed: 1.8ms preprocess, 8.6ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.5ms
Speed: 1.5ms preprocess, 11.5ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 12.9ms
Speed: 1.5ms preprocess, 12.9ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 13.9ms
Speed: 1.8ms preprocess, 13.9ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.7ms
Speed: 1.6ms preprocess, 9.7ms inferen